# motive: Analyse the scraped news data  

In [271]:
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import re
import html
import unicodedata 
plt.style.use("dark_background")

In [272]:
data = pd.read_csv(r"../data/BBCNewsScraper_at_2021-06-06_cached.csv")
data.head()

,url,title,raw_text,author,publish_datetime
0,https://www.bbc.com/news/uk-wales-57146299,Domestic abuse: Swansea man jailed for murderi...,A man has been jailed for life for battering h...,Rhys WilliamsBBC News,2021-05-17T16:29:13.000Z
1,https://www.bbc.com/news/world-asia-india-5677...,Covid-19: How India failed to prevent a deadly...,"In early March, India's health minister Harsh ...",tik BiswasIndia correspondent,2021-04-18T23:01:17.000Z
2,https://www.bbc.com/news/uk-england-northampto...,"Northampton blaze 'cruel blow', say firm's owners",Two friends who spent 10 years building up the...,bbc,2021-05-02T12:13:52.000Z
3,https://www.bbc.com/news/world-us-canada-57209148,Einstein handwritten letter with equation fetc...,A letter written by Albert Einstein containing...,bbc,2021-05-22T03:55:50.000Z
4,https://www.bbc.com/news/world-us-canada-57232694,Florida high school alters 80 'immodest' yearb...,A Florida high school is facing a backlash for...,bbc,2021-05-24T20:04:32.000Z


## preprocess the text data

In [277]:
data.loc[4, "raw_text"]

'A Florida high school is facing a backlash for altering the yearbook photos of 80 female students to add clothing to their chests and shoulders. The school district told local media the changes were made to ensure the photos met the dress code, which says girls\' shirts must be "modest". But critics pointed to yearbook photos of male students left unedited despite violating the same standards.  The digital alterations were made without permission, the students say. Bartram Trail High School\'s yearbook co-ordinator - a female teacher - made the decision to edit the photos after determining they had violated the dress code, the St Johns County school district said. A disclaimer on Bartram Trail\'s website warns students that their yearbook photos may be digitally edited to be consistent with the dress code.  But some students have called the alterations distressing and accused the school of sexism. "The double standard in the yearbook is more so that they looked at our body and thought